In [23]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [25]:
# !pip install tensorflow-addons

In [26]:
# import tensorflow as tf
# import tensorflow_addons as tfa

In [27]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [28]:
# The number of steps within one time segment
TIME_PERIODS = 1
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 1
N_FEATURES = 10

In [29]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [30]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [31]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [32]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [33]:
 pd.options.display.float_format = "{:,.5f}".format

In [34]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [35]:
verbose, epochs, batch_size = 0, 100, 500

In [36]:
def step_decay(epoch):
   initial_lrate = 0.1
   drop = 0.1
   epochs_drop = 32.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [45]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]
test_seg_acc=[]

In [46]:
import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/sub_wise_process_TT/'

In [47]:
def create_segmented_labels(df, time_steps,step):
    labels = []
    for i in range(0, len(df) - time_steps, step):
      label = stats.mode(df[i: i + time_steps])[0][0]
      labels.append(label)
    labels = np.asarray(labels)
    return labels

In [48]:
# weight_decay=0.0001

In [49]:
# extend_with_decoupled_weight_decay(tf.keras.optimizers.SGD,
#                                    weight_decay=weight_decay)

In [50]:
for i in range(1,28):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    pd.options.display.float_format = "{:,.5f}".format
    df_Train.head(5)
    pd.options.display.float_format = "{:,.5f}".format
    df_Test.head(5)
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
    df_Test.head(5)    
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 1, 10
    n_depth=1
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=l2(1e-04), kernel_initializer="he_normal",strides=(1,1), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    model.add(Activation('relu'))
    # model.add(Dropout(0.5))
    model.add(Conv2D(filters=64, kernel_size=(3,3),padding="same", kernel_regularizer=l2(1e-04),kernel_initializer="he_normal",strides=(1,1)))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    model.add(Activation('relu'))
    # model.add(Dropout(0.5))
    model.add(LocallyConnected2D(64, kernel_size=(1, 1), kernel_regularizer=l2(1e-04),kernel_initializer="he_normal",strides=(1,1),padding='valid'))
#     model.add(Conv2D(64, kernel_size=(1, 1), kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=(1,1),padding='valid'))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    model.add(Activation('relu'))
    # model.add(Dropout(0.5))
    model.add(LocallyConnected2D(64, kernel_size=(1, 1), kernel_regularizer=l2(1e-04),kernel_initializer="he_normal",strides=(1,1),padding='valid'))
#     model.add(Conv2D(64, kernel_size=(1, 1), kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=(1,1),padding='valid'))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512, kernel_initializer="he_normal"))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, kernel_initializer="he_normal"))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, kernel_initializer="he_normal"))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
    model.add(Activation('relu'))
    # model.add(Dense(512, activation='tanh'))
    model.add(Dense(n_outputs, activation='softmax'))
#     adam=optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
    sgd=optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/CNN20X10/checkpoint.hdf5'
    pretrained_path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/Trained_model/checkpoint.hdf5'
#     model.load_weights(pretrained_path) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/CNN20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('Average_instantaneous_test_accuracy_for_27subjects')
    statistics.mean(test_acc)
    scores=model.predict(x_test)
    print("scores shape:", scores.shape)
    max_scores = np.argmax(scores, axis=1)
    print('scores_shape',max_scores.shape)
    new_time=20
    new_step=20
    max_scores=create_segmented_labels(max_scores,new_time,new_step)
    print('max_score_shape',max_scores.shape)
    x_test_seg=create_segmented_labels(y_test,new_time,new_step)
    print('x_test_seg_shape',x_test_seg.shape)
    report=classification_report(max_scores, x_test_seg, output_dict='True')
    test_seg_accuracy=report['accuracy']
    test_seg_acc.append(test_seg_accuracy)
    print(test_seg_acc)
    print('Average_test_accuracy_for_27subjects_with_200ms_majority_voting_window')
    statistics.mean(test_seg_acc)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 182000

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 78000

x_train shape:  (181999, 1, 10)
181999 training samples
y_train shape:  (181999,)
num_time_periods 1
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (1, 10)
input_shape: (1, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_trai

362/364 [============================>.] - ETA: 0s - loss: 0.6885 - accuracy: 0.7954
Epoch 00009: val_accuracy did not improve from 0.70710
364/364 [==============================] - 5s 15ms/step - loss: 0.6885 - accuracy: 0.7955 - val_loss: 1.1851 - val_accuracy: 0.7030 - lr: 0.1000
Epoch 10/100
362/364 [============================>.] - ETA: 0s - loss: 0.6628 - accuracy: 0.8039
Epoch 00010: val_accuracy improved from 0.70710 to 0.70742, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/CNN20X10/checkpoint.hdf5
364/364 [==============================] - 5s 14ms/step - loss: 0.6629 - accuracy: 0.8038 - val_loss: 1.1477 - val_accuracy: 0.7074 - lr: 0.1000
Epoch 11/100
362/364 [============================>.] - ETA: 0s - loss: 0.6395 - accuracy: 0.8120
Epoch 00011: val_accuracy did not improve from 0.70742
364/364 [==============================] - 5s 13ms/step - loss: 0.6395 - accuracy: 0.8120 - val_loss: 1.2119 - val_accuracy: 0.7003 - lr: 0.1000
Epo

359/364 [============================>.] - ETA: 0s - loss: 0.3501 - accuracy: 0.9109
Epoch 00034: val_accuracy improved from 0.72141 to 0.72259, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/Geng_2017/CNN20X10/checkpoint.hdf5
364/364 [==============================] - 5s 13ms/step - loss: 0.3502 - accuracy: 0.9108 - val_loss: 1.3059 - val_accuracy: 0.7226 - lr: 0.0100
Epoch 35/100
363/364 [============================>.] - ETA: 0s - loss: 0.3461 - accuracy: 0.9123
Epoch 00035: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 13ms/step - loss: 0.3460 - accuracy: 0.9124 - val_loss: 1.3227 - val_accuracy: 0.7213 - lr: 0.0100
Epoch 36/100
362/364 [============================>.] - ETA: 0s - loss: 0.3408 - accuracy: 0.9134
Epoch 00036: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 14ms/step - loss: 0.3407 - accuracy: 0.9134 - val_loss: 1.3259 - val_accuracy: 0.7210 - lr: 0.0100
Epo

362/364 [============================>.] - ETA: 0s - loss: 0.2784 - accuracy: 0.9263
Epoch 00061: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 13ms/step - loss: 0.2784 - accuracy: 0.9263 - val_loss: 1.4397 - val_accuracy: 0.7165 - lr: 0.0100
Epoch 62/100
362/364 [============================>.] - ETA: 0s - loss: 0.2792 - accuracy: 0.9255
Epoch 00062: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 13ms/step - loss: 0.2792 - accuracy: 0.9255 - val_loss: 1.4459 - val_accuracy: 0.7159 - lr: 0.0100
Epoch 63/100
364/364 [==============================] - ETA: 0s - loss: 0.2768 - accuracy: 0.9264
Epoch 00063: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 13ms/step - loss: 0.2768 - accuracy: 0.9264 - val_loss: 1.4592 - val_accuracy: 0.7153 - lr: 0.0100
Epoch 64/100
364/364 [==============================] - ETA: 0s - loss: 0.2663 - accuracy: 0.9295
Epoch 00064: val_accu

Epoch 89/100
361/364 [============================>.] - ETA: 0s - loss: 0.2471 - accuracy: 0.9360
Epoch 00089: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 13ms/step - loss: 0.2472 - accuracy: 0.9360 - val_loss: 1.4495 - val_accuracy: 0.7190 - lr: 0.0010
Epoch 90/100
362/364 [============================>.] - ETA: 0s - loss: 0.2494 - accuracy: 0.9350
Epoch 00090: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 13ms/step - loss: 0.2493 - accuracy: 0.9350 - val_loss: 1.4413 - val_accuracy: 0.7193 - lr: 0.0010
Epoch 91/100
364/364 [==============================] - ETA: 0s - loss: 0.2484 - accuracy: 0.9356
Epoch 00091: val_accuracy did not improve from 0.72259
364/364 [==============================] - 5s 13ms/step - loss: 0.2484 - accuracy: 0.9356 - val_loss: 1.4483 - val_accuracy: 0.7199 - lr: 0.0010
Epoch 92/100
364/364 [==============================] - ETA: 0s - loss: 0.2471 - accuracy: 0.9364
Epoch 00

...

x_train shape:  (181999, 1, 10)
181999 training samples
y_train shape:  (181999,)
num_time_periods 1
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (1, 10)
input_shape: (1, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (181999, 52)
x_test shape:  (77999, 1, 10)
77999 testing samples
y_test shape:  (77999,)
x_train shape:  (181999, 1, 10, 1)
x_test shape:  (77999, 1, 10, 1)
n_outputs 52
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)      

Epoch 11/100
359/364 [============================>.] - ETA: 0s - loss: 0.7910 - accuracy: 0.7671
Epoch 00011: val_accuracy did not improve from 0.62017
364/364 [==============================] - 5s 13ms/step - loss: 0.7919 - accuracy: 0.7669 - val_loss: 1.8162 - val_accuracy: 0.5988 - lr: 0.1000
Epoch 12/100
361/364 [============================>.] - ETA: 0s - loss: 0.7695 - accuracy: 0.7746
Epoch 00012: val_accuracy did not improve from 0.62017
364/364 [==============================] - 5s 13ms/step - loss: 0.7696 - accuracy: 0.7746 - val_loss: 1.8179 - val_accuracy: 0.6138 - lr: 0.1000
Epoch 13/100
364/364 [==============================] - ETA: 0s - loss: 0.7492 - accuracy: 0.7823
Epoch 00013: val_accuracy did not improve from 0.62017
364/364 [==============================] - 5s 12ms/step - loss: 0.7492 - accuracy: 0.7823 - val_loss: 1.8051 - val_accuracy: 0.6193 - lr: 0.1000
Epoch 14/100
363/364 [============================>.] - ETA: 0s - loss: 0.7320 - accuracy: 0.7870
Epoch 00

Epoch 37/100
364/364 [==============================] - ETA: 0s - loss: 0.4106 - accuracy: 0.8937
Epoch 00037: val_accuracy did not improve from 0.63033
364/364 [==============================] - 5s 13ms/step - loss: 0.4106 - accuracy: 0.8937 - val_loss: 2.1437 - val_accuracy: 0.6280 - lr: 0.0100
Epoch 38/100
361/364 [============================>.] - ETA: 0s - loss: 0.4075 - accuracy: 0.8950
Epoch 00038: val_accuracy did not improve from 0.63033
364/364 [==============================] - 4s 12ms/step - loss: 0.4076 - accuracy: 0.8950 - val_loss: 2.1487 - val_accuracy: 0.6289 - lr: 0.0100
Epoch 39/100
359/364 [============================>.] - ETA: 0s - loss: 0.4061 - accuracy: 0.8951
Epoch 00039: val_accuracy did not improve from 0.63033
364/364 [==============================] - 5s 13ms/step - loss: 0.4060 - accuracy: 0.8952 - val_loss: 2.1546 - val_accuracy: 0.6280 - lr: 0.0100
Epoch 40/100
362/364 [============================>.] - ETA: 0s - loss: 0.4011 - accuracy: 0.8953
Epoch 00

362/364 [============================>.] - ETA: 0s - loss: 0.3241 - accuracy: 0.9143
Epoch 00064: val_accuracy did not improve from 0.63061
364/364 [==============================] - 5s 13ms/step - loss: 0.3238 - accuracy: 0.9145 - val_loss: 2.3018 - val_accuracy: 0.6260 - lr: 0.0010
Epoch 65/100
363/364 [============================>.] - ETA: 0s - loss: 0.3190 - accuracy: 0.9156
Epoch 00065: val_accuracy did not improve from 0.63061
364/364 [==============================] - 5s 13ms/step - loss: 0.3191 - accuracy: 0.9155 - val_loss: 2.2799 - val_accuracy: 0.6248 - lr: 0.0010
Epoch 66/100
361/364 [============================>.] - ETA: 0s - loss: 0.3197 - accuracy: 0.9157
Epoch 00066: val_accuracy did not improve from 0.63061
364/364 [==============================] - 5s 13ms/step - loss: 0.3199 - accuracy: 0.9156 - val_loss: 2.3008 - val_accuracy: 0.6255 - lr: 0.0010
Epoch 67/100
363/364 [============================>.] - ETA: 0s - loss: 0.3164 - accuracy: 0.9162
Epoch 00067: val_accu

KeyboardInterrupt: 

In [ ]:
print('Average_instantaneous_test_accuracy_for_27subjects')
statistics.mean(test_acc)

In [ ]:
print('Average_test_accuracy_for_27subjects_with_200ms_majority_voting_window')
statistics.mean(test_seg_acc)